In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/ATIML/

/content/drive/MyDrive/Colab Notebooks/ATIML


In [ ]:
import pickle
import time

import os
import string
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import spacy

from gensim.models import Word2Vec
import nltk
import re
import pandas as pd

from nltk import ngrams

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from pprint import pprint

import numpy as np

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans, KMeans

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#@title Default title text
rootdir = '/content/drive/MyDrive/ATML/Datasets/TREC_4_5/'
iterable_lines = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
      #print("Parsing through:", os.path.join(subdir, file))
      f = open((str(subdir)+"/"+str(file)), 'r', encoding="ISO-8859-1")
      lines = f.read()
      f.close()
      iterable_lines += [lines]

In [ ]:
#@title Default title text
processed_articles = []
for i in iterable_lines:
  processed_article = i.lower()
  processed_article = re.sub(r"\[.*?\]", '', processed_article)
  processed_article = re.sub(r"https?://\S+|www\.\S+", '', processed_article)
  processed_article = re.sub(r"<.*?>+", '', processed_article)
  processed_article = re.sub(r"[%s]" % re.escape(string.punctuation), '', processed_article)
  processed_article = re.sub(r"\n", ' ', processed_article)
  processed_article = re.sub(r"\w*\d\w*", '', processed_article)
  processed_article = re.sub(r" +", ' ', processed_article)
  processed_articles += [processed_article]

In [ ]:
#@title Default title text
tokenized_processed_articles = []

for i in range(len(processed_articles)):
  tokenized_processed_articles += [nltk.word_tokenize(processed_articles[i])]

In [ ]:
import pandas as pd
tokenized_lemm_articles = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ATIML/lemm_doc.pkl")

In [ ]:
print(len(tokenized_lemm_articles))

2322


In [ ]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_lemm_articles)]

In [ ]:
model = Doc2Vec(tagged_data, window=2, vector_size=256, min_count=3, workers=4, epochs = 100)
# vector_size=100

In [ ]:
import pickle
with open('doc2vecmodel.pkl','wb') as f:
  pickle.dump(model, f)

In [ ]:
similar_docs = model.docvecs.most_similar(1121, topn=3)

In [ ]:
print(similar_docs)

[(1099, 0.6787018179893494), (1039, 0.6692175269126892), (568, 0.6618386507034302)]


In [ ]:
for k in similar_docs:
  print(k[0])

1099
1039
568


In [ ]:
for i in range(len(similar_docs)):
    print(
            "Document:"+str(similar_docs[i][0]) + " Similarity Score:"+str(
                similar_docs[i][1]
            ))
    print("************************************************")
    print("************************************************")

Document:568 Similarity Score:0.6903170943260193
************************************************
************************************************
Document:1039 Similarity Score:0.6863768100738525
************************************************
************************************************
Document:1238 Similarity Score:0.6849675178527832
************************************************
************************************************


In [ ]:
for k in similar_docs:
        print("Inside k")
        doc_number = k[0]
        print("doc_number {}".format(doc_number))
        print("Key phrases for document "+str(k[0])+":")

        print("************************************************")

Inside k
doc_number 1099
Key phrases for document 1099:
************************************************
Inside k
doc_number 1039
Key phrases for document 1039:
************************************************
Inside k
doc_number 568
Key phrases for document 568:
************************************************


In [ ]:
import pickle
with open("my_doc2vec_model.pkl", 'rb') as f:  # the saved model that has been trained on version 3.8.3 (I can't provide - NDA)
    model = pickle.dump(model)
model.docvecs.most_similar(id)  # or model.dv.most_similar(id)

FileNotFoundError: ignored

In [ ]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/content/drive/MyDrive/Colab Notebooks/ATIML/my_doc2vec_model")
model.save(fname)


In [ ]:
model = Doc2Vec.load(fname)

In [ ]:
model.corpus_count

2322

In [ ]:
model.vector_size

256

In [ ]:
def get_init_constraints(tokenized_lemm_articles):
  ml_init_constraints = []
  cl_init_constraints = []
  for doc_id in range(len(tokenized_lemm_articles)):
    # Compare and print the most-similar, disimilar document
    sim_doc = model.docvecs.most_similar(doc_id, topn=len(model.docvecs))[0]
    if sim_doc[1] >= 0.8:
      ml_init_constraints += [[doc_id, sim_doc]]
    dissim_doc = model.docvecs.most_similar(doc_id, topn=len(model.docvecs))[-1]
    if dissim_doc[1] <= 0.1:
      cl_init_constraints += [[doc_id, dissim_doc]]
  return ml_init_constraints, cl_init_constraints



In [ ]:
#get the similiar and disimilar docs by sorting based on similarity values
def sort_on_similarity(initial_constraints):
  return sorted(initial_constraints, key=lambda tup: tup[1][1], reverse=True)

#Extracting instances/docs after sorting based on similarity
def extract_docs(ml_sim_sorted):
  li=[]
  for item in ml_sim_sorted:
    li.append(((item[0], item[1][0])))
  return li

#sorting each tuple inside the list to follow ascending order
#this is done to findout if there are duplicate tuples
def inner_tuple_sort(extracted_docs):
  sort_tuples = []
  for element in extracted_docs:
    sort_tuples.append(tuple(sorted(element)))
  return sort_tuples

#getting unique tuples preserving the order of tuples
def unique(sequence):
  seen = set()
  return [x for x in sequence if not (tuple(x) in seen or seen.add(tuple(x)))]


In [ ]:
#initialize N
N = 25

ml_init_constraints , cl_init_constraints = get_init_constraints(tokenized_lemm_articles)

ml_sim_sorted = sort_on_similarity(ml_init_constraints)
extracted_ml_docs = extract_docs(ml_sim_sorted)
sorted_ml_tuples = inner_tuple_sort(extracted_ml_docs)

final_ml_constraints = unique(sorted_ml_tuples)[:N]

cl_sim_sorted = sort_on_similarity(cl_init_constraints)
extracted_cl_docs = extract_docs(cl_sim_sorted)
sorted_cl_tuples = inner_tuple_sort(extracted_cl_docs)

final_cl_constraints = unique(sorted_cl_tuples)[:N]

print(final_ml_constraints)
print(final_cl_constraints)


[(494, 495), (495, 496), (494, 498), (33, 264), (1199, 1232), (194, 195), (928, 979), (1382, 1508), (1296, 1416), (1515, 1526), (986, 989), (34, 39), (247, 334), (264, 270), (511, 516), (183, 343), (1373, 1452), (188, 351), (1232, 1339), (669, 733), (280, 285), (34, 38), (326, 420), (1258, 1353), (173, 326)]
[(678, 1520), (495, 995), (157, 1139), (681, 939), (209, 1311), (559, 1441), (791, 1464), (75, 978), (498, 995), (76, 1050), (65, 1480), (898, 2236), (4, 494), (602, 680), (169, 402), (687, 2083), (60, 734), (496, 995), (37, 1106), (708, 1480), (960, 1774), (1349, 2108), (314, 564), (145, 999), (299, 1125)]


In [ ]:
print(final_ml_constraints)


[(495, 498), (494, 495), (496, 498), (33, 264), (1199, 1232), (183, 343), (928, 979), (1232, 1339), (247, 334), (1017, 1025), (34, 39), (34, 38), (1373, 1452), (194, 195), (173, 326), (1004, 1007), (1343, 1443), (1296, 1308), (56, 58), (1339, 1381), (188, 351), (114, 189), (189, 349), (669, 733), (264, 270), (497, 498), (194, 359), (1320, 1351), (1346, 1381), (33, 113), (193, 194), (1353, 1381), (280, 285), (1787, 1857), (120, 277), (1296, 1416), (107, 183), (1258, 1356), (1515, 1518), (1346, 1386), (1355, 1515), (986, 989), (997, 1000), (2116, 2183), (1308, 1468), (35, 41), (503, 505), (1787, 1869), (1343, 1570), (274, 275)]


In [ ]:
import pickle
with open('final_ml_constraints.pkl','wb') as f:
  pickle.dump(final_ml_constraints, f)
with open('final_cl_constraints.pkl','wb') as f:
  pickle.dump(final_cl_constraints, f)

In [ ]:
import pandas as pd
final_ml_constraints = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ATIML/final_ml_constraints.pkl")
final_cl_constraints = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/ATIML/final_ml_constraints.pkl")

In [ ]:
#word2vec uses cosine sim between docs.Crosschecking here
cosine_similarity(model.docvecs[0].reshape(1, -1), model.docvecs[2007].reshape(1, -1))

array([[0.59106445]], dtype=float32)